In [64]:
# timerange202401-202410data
# t0, t1dataprocess, read./visualization/assets/data/xxx/rankxxx.csvdata
# rankdict, keycommunity, valuecommunitynodecolumnid
# target: t0->t1communityexists, , , , , .generatecommunity events
# communityevent, event, communityevent, event, # event: t0communityt1communitynodejaccardcommunitytype(, ), , ; communityevent, event, columndelete

# define, communityjaccard
# t0community, community, t1communityjaccard, jaccardcommunity, jaccardcommunitytype, , , , ; , ��, # jaccard: 0, 0.8, 0.95, 0.1-0.8
# , , , t0, t1Removedeventcommunity, community, # t0community, community, typet1community(: t0t1community/t0community), , exists0.8, type, ., 0.8type, .
# ���type, t0��t1community, ���t1community, t0community(: t0t1community/t1community), , 0.8type, .


In [65]:
import pandas as pd
import numpy as np
import os
import json
from collections import defaultdict


In [66]:
def calculate_jaccard(set1, set2):
 """Jaccard"""
 intersection = len(set1.intersection(set2))
 union = len(set1.union(set2))
 return intersection / union if union > 0 else 0

def calculate_overlap(set1, set2, denominator_set):
 """: / denominator_set"""
 intersection = len(set1.intersection(set2))
 denominator = len(denominator_set)
 return intersection / denominator if denominator > 0 else 0


In [67]:
def load_community_data(date):
 """communitydata"""
 file_path = f'../visualization/assets/data/{date}/handle/rank{date}.csv'
 if not os.path.exists(file_path):
 print(f"File not found: {file_path}")
 return None

 df = pd.read_csv(file_path)

 # community {communityID: node}
 communities = {}
 for _, row in df.iterrows():
 comm_id = row['community']
 node_id = row['id']

 if comm_id not in communities:
 communities[comm_id] = set()
 communities[comm_id].add(node_id)

 return communities


In [68]:
def analyze_community_events(t0_date, t1_date):
 """timecommunity events"""
 print(f"Analyzing events between {t0_date} and {t1_date}")

 # timecommunitydata
 t0_communities = load_community_data(t0_date)
 t1_communities = load_community_data(t1_date)

 if t0_communities is None or t1_communities is None:
 print("Missing community data, skipping analysis")
 return None

 # eventcommunity
 t0_assigned = set()
 t1_assigned = set()

 # event
 events = []

 # communityt0community()
 t0_communities_sorted = sorted(
 [(comm_id, nodes) for comm_id, nodes in t0_communities.items()],
 key=lambda x: len(x[1]),
 reverse=True
 )

 # communityt1community()
 t1_communities_sorted = sorted(
 [(comm_id, nodes) for comm_id, nodes in t1_communities.items()],
 key=lambda x: len(x[1]),
 reverse=True
 )

 # : , , - communityprocess
 for t0_comm_id, t0_nodes in t0_communities_sorted:
 # skipcommunity(process)
 if len(t0_nodes) < 100:
 continue
  if t0_comm_id in t0_assigned:
 continue

 best_match = None
 best_jaccard = 0

 for t1_comm_id, t1_nodes in t1_communities_sorted:
 if t1_comm_id in t1_assigned:
 continue

 jaccard = calculate_jaccard(t0_nodes, t1_nodes)

 if jaccard > best_jaccard:
 best_jaccard = jaccard
 best_match = t1_comm_id

 event_type = None
 target_comm = None

 # Jaccardeventtype
 if best_jaccard == 0:
 event_type = ""
 elif best_jaccard >= 0.9:
 event_type = ""
 target_comm = best_match
 t1_assigned.add(best_match)
 elif best_jaccard >= 0.7:
 # node
 t1_size = len(t1_communities[best_match])
 t0_size = len(t0_nodes)

 if t1_size > t0_size:
 event_type = ""
 else:
 event_type = "���"

 target_comm = best_match
 t1_assigned.add(best_match)
 else:
 # process,  continue

 events.append({
 "source_date": t0_date,
 "source_community": t0_comm_id,
 "target_date": t1_date,
 "target_community": target_comm,
 "event_type": event_type,
 "jaccard": best_jaccard
 })

 t0_assigned.add(t0_comm_id)

 # eventcommunity,  unassigned_t0 = [(comm_id, nodes) for comm_id, nodes in t0_communities_sorted
 if comm_id not in t0_assigned]
 unassigned_t1 = [(comm_id, nodes) for comm_id, nodes in t1_communities_sorted
 if comm_id not in t1_assigned]

 # : process - communityprocess, community
 for t0_comm_id, t0_nodes in [x for x in unassigned_t0 if len(x[1]) >= 100]:
 # t1community
 overlap_scores = []

 for t1_comm_id, t1_nodes in unassigned_t1:
 overlap = calculate_overlap(t0_nodes, t1_nodes, t0_nodes)
 if overlap > 0:
 overlap_scores.append((t1_comm_id, overlap, len(t1_nodes)))

 if not overlap_scores:
 continue

 # , t1community(community)
 overlap_scores.sort(key=lambda x: (x[1], x[2]), reverse=True)

 # checkevent -  high_overlap_targets = [t1_id for t1_id, score, _ in overlap_scores if score > 0.3]

 if len(high_overlap_targets) > 1:
 # targetcommunity
 cumulative_overlap = 0
 split_targets = []
 total_target_size = 0
 source_size = len(t0_nodes)

 for t1_id, score, t1_size in overlap_scores:
 if t1_id in t1_assigned:
 continue

 cumulative_overlap += score
 split_targets.append(t1_id)
 total_target_size += t1_size
 t1_assigned.add(t1_id)

 # check: target1.5
 size_ratio = total_target_size / source_size
 if cumulative_overlap >= 0.8 and size_ratio <= 1.5:
 break
  # , , add
 if size_ratio > 1.5:
 # addcommunity
 t1_assigned.remove(t1_id)
 split_targets.pop()
 total_target_size -= t1_size
 cumulative_overlap -= score
 break

 if cumulative_overlap >= 0.5 and split_targets and (total_target_size / source_size <= 1.5):
 events.append({
 "source_date": t0_date,
 "source_community": t0_comm_id,
 "target_date": t1_date,
 "target_community": split_targets,
 "event_type": "",
 "overlap_score": cumulative_overlap,
 "size_ratio": total_target_size / source_size
 })
 t0_assigned.add(t0_comm_id)

 # eventcommunity
 unassigned_t0 = [(comm_id, nodes) for comm_id, nodes in t0_communities_sorted
 if comm_id not in t0_assigned]
 unassigned_t1 = [(comm_id, nodes) for comm_id, nodes in t1_communities_sorted
 if comm_id not in t1_assigned]

 # : process - targetcommunity, community
 for t1_comm_id, t1_nodes in [x for x in unassigned_t1 if len(x[1]) >= 100]:
 # t0community
 overlap_scores = []

 for t0_comm_id, t0_nodes in unassigned_t0:
 overlap = calculate_overlap(t1_nodes, t0_nodes, t1_nodes)
 if overlap > 0:
 overlap_scores.append((t0_comm_id, overlap, len(t0_nodes)))

 if not overlap_scores:
 continue

 # , community
 overlap_scores.sort(key=lambda x: (x[1], x[2]), reverse=True)

 # checkevent
 high_overlap_sources = [t0_id for t0_id, score, _ in overlap_scores if score > 0.3]

 if len(high_overlap_sources) > 1:
 # community
 cumulative_overlap = 0
 merge_sources = []
 total_source_size = 0
 target_size = len(t1_nodes)

 for t0_id, score, t0_size in overlap_scores:
 if t0_id in t0_assigned:
 continue

 cumulative_overlap += score
 merge_sources.append(t0_id)
 total_source_size += t0_size
 t0_assigned.add(t0_id)

 # check: target1.5
 size_ratio = total_source_size / target_size
 if cumulative_overlap >= 0.8 and size_ratio <= 1.5:
 break
  # , , add
 if size_ratio > 1.5:
 # addcommunity
 t0_assigned.remove(t0_id)
 merge_sources.pop()
 total_source_size -= t0_size
 cumulative_overlap -= score
 break

 if cumulative_overlap >= 0.5 and merge_sources and (total_source_size / target_size <= 1.5):
 events.append({
 "source_date": t0_date,
 "source_community": merge_sources,
 "target_date": t1_date,
 "target_community": t1_comm_id,
 "event_type": "",
 "overlap_score": cumulative_overlap,
 "size_ratio": total_source_size / target_size
 })
 t1_assigned.add(t1_comm_id)

 # : processcommunity(<100 nodes)-  small_t0_communities = [(comm_id, nodes) for comm_id, nodes in unassigned_t0 if len(nodes) < 100]
  # processcommunity
 for t0_comm_id, t0_nodes in small_t0_communities:
 if t0_comm_id in t0_assigned:
 continue
  # t1community(event)
 overlap_scores = []
  for t1_comm_id, t1_nodes in t1_communities_sorted:
 overlap = calculate_overlap(t0_nodes, t1_nodes, t0_nodes)
 if overlap > 0:
 overlap_scores.append((t1_comm_id, overlap, len(t1_nodes)))
  if not overlap_scores:
 continue
  #  overlap_scores.sort(key=lambda x: x[1], reverse=True)
  # checkcondition
 best_match_id, best_overlap, best_match_size = overlap_scores[0]
  # condition: targetcommunitycommunity()
 if best_overlap >= 0.5 and best_match_size >= len(t0_nodes) * 1.5:
 events.append({
 "source_date": t0_date,
 "source_community": t0_comm_id,
 "target_date": t1_date,
 "target_community": best_match_id,
 "event_type": "",
 "overlap_score": best_overlap,
 "size_ratio": best_match_size / len(t0_nodes)
 })
 t0_assigned.add(t0_comm_id)
  # processcommunity
 small_t1_communities = [(comm_id, nodes) for comm_id, nodes in unassigned_t1 if len(nodes) < 100]
  for t1_comm_id, t1_nodes in small_t1_communities:
 if t1_comm_id in t1_assigned:
 continue
  # t0community(event)
 overlap_scores = []
  for t0_comm_id, t0_nodes in t0_communities_sorted:
 if t0_comm_id in t0_assigned: # skipt0community
 continue
  overlap = calculate_overlap(t1_nodes, t0_nodes, t1_nodes)
 if overlap > 0:
 overlap_scores.append((t0_comm_id, overlap, len(t0_nodes)))
  if not overlap_scores:
 continue
  #  overlap_scores.sort(key=lambda x: x[1], reverse=True)
  # checkcondition
 best_match_id, best_overlap, best_match_size = overlap_scores[0]
  # condition: communitytargetcommunity()
 if best_overlap >= 0.5 and best_match_size >= len(t1_nodes) * 1.5:
 events.append({
 "source_date": t0_date,
 "source_community": best_match_id,
 "target_date": t1_date,
 "target_community": t1_comm_id,
 "event_type": "",
 "overlap_score": best_overlap,
 "size_ratio": best_match_size / len(t1_nodes)
 })
 t0_assigned.add(best_match_id)
 t1_assigned.add(t1_comm_id)

 # checkcommunity (t1eventcommunity)
 for t1_comm_id, t1_nodes in sorted(
 [(comm_id, nodes) for comm_id, nodes in t1_communities.items() if comm_id not in t1_assigned],
 key=lambda x: len(x[1]),
 reverse=True
 ):
 events.append({
 "source_date": t0_date,
 "source_community": None,
 "target_date": t1_date,
 "target_community": t1_comm_id,
 "event_type": "",
 "jaccard": 0
 })

 return events


In [69]:
def generate_all_events():
 """generatecommunity events"""
 all_events = []
 months = list(range(202401, 202411))

 for i in range(len(months) - 1):
 t0 = months[i]
 t1 = months[i + 1]

 events = analyze_community_events(t0, t1)
 if events:
 all_events.extend(events)

 # saveeventdata
 output_dir = '../visualization/assets/data/events'
 os.makedirs(output_dir, exist_ok=True)

 # JSONcolumnformatsave
 events_json = []
 for event in all_events:
 event_copy = event.copy()
 # column
 if isinstance(event_copy.get('source_community'), list) and len(event_copy['source_community']) > 0:
 if isinstance(event_copy['source_community'][0], set):
 event_copy['source_community'] = [list(comm) for comm in event_copy['source_community']]
 if isinstance(event_copy.get('target_community'), list) and len(event_copy['target_community']) > 0:
 if isinstance(event_copy['target_community'][0], set):
 event_copy['target_community'] = [list(comm) for comm in event_copy['target_community']]
 events_json.append(event_copy)

 with open(f'{output_dir}/community_events.json', 'w') as f:
 json.dump(events_json, f, indent=2)

 # generateCSVformat
 events_df = pd.DataFrame(all_events)
 events_df.to_csv(f'{output_dir}/community_events.csv', index=False)

 print(f"Generated {len(all_events)} community events")
 return all_events


In [70]:
# generateevent
all_events = generate_all_events()


Analyzing events between 202401 and 202402
Analyzing events between 202402 and 202403
Analyzing events between 202403 and 202404
Analyzing events between 202404 and 202405
Analyzing events between 202405 and 202406
Analyzing events between 202406 and 202407
Analyzing events between 202407 and 202408
Analyzing events between 202408 and 202409
Analyzing events between 202409 and 202410
Generated 404 community events


In [71]:
# outputeventtype
if all_events:
 event_counts = defaultdict(int)
 for event in all_events:
 event_counts[event['event_type']] += 1

 print("eventtype:")
 for event_type, count in event_counts.items():
 print(f"{event_type}: {count}")


eventtype:
���: 8
: 115
: 29
: 237
: 5
: 10


In [72]:
# communitypath
def visualize_community_evolution(community_id, start_date):
 """
 communitypath
 """
 if not all_events:
 print("No events data available")
 return

 evolution_path = []
 current_communities = [community_id]
 current_date = start_date

 while current_communities and current_date < 202410:
 next_date = current_date + 1
 if next_date % 100 > 12: # process
 next_date = (next_date // 100 + 1) * 100 + 1

 step_events = []
 next_communities = []

 for current_comm in current_communities:
 # communitytimeevent
 for event in all_events:
 if event['source_date'] == current_date and event['source_community'] == current_comm:
 step_events.append(event)
 if event['event_type'] != '':
 if isinstance(event['target_community'], list):
 next_communities.extend(event['target_community'])
 else:
 next_communities.append(event['target_community'])

 if step_events:
 evolution_path.append({
 'from_date': current_date,
 'to_date': next_date,
 'events': step_events
 })

 current_communities = next_communities
 current_date = next_date

 # outputpath
 if evolution_path:
 print(f"community {community_id} path ( {start_date}):")
 for step in evolution_path:
 print(f" {step['from_date']} {step['to_date']}:")
 for event in step['events']:
 print(f" {event['event_type']} -> {event['target_community']}")
 else:
 print(f"community {community_id} path")

 return evolution_path


In [73]:
# : 202401community0
visualize_community_evolution(1, 202401)


community 1 path ( 202401):
 202401 202402:
 ��� -> 10
 202402 202403:
 -> 9
 202403 202404:
 -> 4
 202404 202405:
 -> 2
 202405 202406:
 ��� -> 14
 202406 202407:
 -> 3
 202407 202408:
 ��� -> 2


[{'from_date': 202401,
 'to_date': 202402,
 'events': [{'source_date': 202401,
 'source_community': 1,
 'target_date': 202402,
 'target_community': 10,
 'event_type': '���',
 'jaccard': 0.7568306010928961}]},
 {'from_date': 202402,
 'to_date': 202403,
 'events': [{'source_date': 202402,
 'source_community': 10,
 'target_date': 202403,
 'target_community': 9,
 'event_type': '',
 'overlap_score': 0.75,
 'size_ratio': 2723.0}]},
 {'from_date': 202403,
 'to_date': 202404,
 'events': [{'source_date': 202403,
 'source_community': 9,
 'target_date': 202404,
 'target_community': 4,
 'event_type': '',
 'overlap_score': 0.75,
 'size_ratio': 3093.25}]},
 {'from_date': 202404,
 'to_date': 202405,
 'events': [{'source_date': 202404,
 'source_community': 4,
 'target_date': 202405,
 'target_community': 2,
 'event_type': '',
 'jaccard': 0.8149967955565051}]},
 {'from_date': 202405,
 'to_date': 202406,
 'events': [{'source_date': 202405,
 'source_community': 2,
 'target_date': 202406,
 'target_communit